In [1]:
 
import numpy as np


def thomas_algorithm(alpha: float, rhs: np.ndarray) -> np.ndarray:
    """Solve T x = rhs, where T = tridiag(alpha, 1, alpha) (non-cyclic).
    
    Args:
        alpha: Off-diagonal coefficient
        rhs: Right-hand side vector
        
    Returns:
        Solution vector x
    """
    n = rhs.size
    a = np.full(n - 1, alpha)  # Lower diagonal
    b = np.ones(n)             # Main diagonal
    c = np.full(n - 1, alpha)  # Upper diagonal

    # Forward sweep
    cp = np.empty_like(c)      # Modified c coefficients
    dp = np.empty_like(rhs)    # Modified right-hand side

    cp[0] = c[0] / b[0]
    dp[0] = rhs[0] / b[0]

    for i in range(1, n - 1):
        denom = b[i] - a[i - 1] * cp[i - 1]
        cp[i] = c[i] / denom
        dp[i] = (rhs[i] - a[i - 1] * dp[i - 1]) / denom

    # Last row
    denom = b[n - 1] - a[n - 2] * cp[n - 2]
    dp[n - 1] = (rhs[n - 1] - a[n - 2] * dp[n - 2]) / denom

    # Backward substitution
    x = np.empty_like(rhs)
    x[-1] = dp[-1]
    for i in range(n - 2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i + 1]

    return x


def fourth_order_periodic_derivative(u: np.ndarray, dx: float) -> np.ndarray:
    """Compute fourth-order compact derivative on a periodic grid (Lele 1992).
    
    Args:
        u: Function values at grid points
        dx: Grid spacing
        
    Returns:
        Derivative approximation
    """
    n = u.size
    alpha = 0.25
    a_coeff = 1.5
    idx = np.arange(n)

    # Compute right-hand side
    rhs = a_coeff * (u[(idx + 1) % n] - u[(idx - 1) % n]) / (2.0 * dx)

    # Basis vectors for periodic correction
    u_vec = np.zeros(n)  # u = α e₁
    u_vec[0] = alpha
    p_vec = np.zeros(n)  # p = α e_N
    p_vec[-1] = alpha

    # Solve systems
    y = thomas_algorithm(alpha, u_vec)
    z = thomas_algorithm(alpha, p_vec)
    w = thomas_algorithm(alpha, rhs)

    # Solve 2x2 system for periodic correction coefficients
    m00 = 1.0 + y[-1]
    m01 = z[-1]
    m10 = y[0]
    m11 = 1.0 + z[0]

    det = m00 * m11 - m01 * m10
    c0 = (m11 * w[-1] - m01 * w[0]) / det
    c1 = (-m10 * w[-1] + m00 * w[0]) / det

    return w - c0 * y - c1 * z


def main() -> None:
    """Demonstrate the fourth-order periodic derivative."""
    n_points = 128
    x = np.linspace(0.0, 2.0 * np.pi, n_points, endpoint=False)
    dx = x[1] - x[0]
    u = np.sin(x)

    du_num = fourth_order_periodic_derivative(u, dx)
    du_ex = np.cos(x)

    l2_error = np.sqrt(np.mean((du_num - du_ex) ** 2))
    max_error = np.max(np.abs(du_num - du_ex))

    print("L2 error            =", f"{l2_error:.3e}")
    print("Maximum point error =", f"{max_error:.3e}")


if __name__ == "__main__":
    main()

 


L2 error            = 2.281e-08
Maximum point error = 3.227e-08
